### Gates features

In [1]:
#CNOTS
def countCXs(label):
    return 2*(countGates(label)-1)
#Gate counting                                                                          g
def countYgates(pauli_label):
    countYgates = sum(map(lambda x : 1 if 'Y' in x else 0, pauli_label))
    return countYgates
def countZgates(pauli_label):
    countZgates = sum(map(lambda x : 1 if 'Z' in x else 0, pauli_label))
    return countZgates
def countXgates(pauli_label):
    countXgates = sum(map(lambda x : 1 if 'X' in x else 0, pauli_label))
    return countXgates
def countGates(pauli_label):
    return countXgates(pauli_label)+countYgates(pauli_label)+countZgates(pauli_label)

In [1]:
-18.90866601597427+18.91287544869148

0.004209432717210859

### Construct Commutator Pool

In [2]:
def commutatorPool(qubitH):
    #construct commutator pool from the Hamiltonian
    pool_H=[WeightedPauliOperator([[1j,qubitH.paulis[i][1]]]) for i in range(len(qubitH.paulis))][1:]
    #commutator between operators
    commutator_pool=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in pool_H for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1]
    labels1=[commutator_pool[i].paulis[0][1].to_label() for i in range(len(commutator_pool))]
    unique_labels,indices=numpy.unique(labels1,return_index=True)
    commutator_pool=[commutator_pool[indices[i]] for i in range(len(indices))]
    commutator_pool_2=[WeightedPauliOperator([[1j,commutator(op1,op2).paulis[0][1]]]) for op1 in commutator_pool for op2 in pool_H if countYgates(commutator(op1,op2).paulis[0][1].to_label())%2==1 and commutator(op1,op2).paulis[0][1].to_label() not in labels1]
    labels2=[commutator_pool_2[i].paulis[0][1].to_label() for i in range(len(commutator_pool_2))]
    unique_labels,indices=numpy.unique(labels2,return_index=True)
    commutator_pool_2=[commutator_pool_2[indices[i]] for i in range(len(indices))]
    def commuting_groups(c_pool):
        groups=[[c_pool[0]]]
        for i in range(1,len(c_pool)):
            flag=0
            for j in range(len(groups)):
                check_commute=[c_pool[i].commute_with(groups[j][l]) for l in range(len(groups[j]))]
                if False in check_commute:
                    continue
                else:
                    flag=1
                    groups[j].append(c_pool[i])
                    break
            if flag==0:
                groups.append([c_pool[i]])
        tot_members=[]        
        for i in range(len(groups)):
            for j in range(len(groups[i])):
                tot_members.append(groups[i][j])
        tot_members=[groups[i][j] for j in range(len(groups[i])) for i in range(len(groups))]
        return tot_members
    #commutator_pool=commuting_groups(commutator_pool)
    #commutator_pool_2=commuting_groups(commutator_pool_2)
    return numpy.array(commutator_pool),numpy.array(commutator_pool_2)        

### Construct qubit UCCSD Pool

In [ ]:
def qubit_UCCSD_pool(U):
    fer_pool_UCCSD=var_form_base.excitation_pool
    qubit_pool_UCCSD=[]
    for i in range(len(fer_pool_UCCSD)):
        for j in range(len(fer_pool_UCCSD[i].paulis)):
            qubit_pool_UCCSD.append(WeightedPauliOperator([[1j,fer_pool_UCCSD[i].paulis[j][1]]]))
    return numpy.array(qubit_pool_UCCSD)

### Compute Commutator between operators

In [2]:
def commutator(A,B):
        B2=WeightedPauliOperator([[-B.paulis[0][0],B.paulis[0][1]]])
        return A.multiply(B).add(B2.multiply(A))

### Construct Hamiltonian from integrals

In [3]:
def qubitOp(h1,h2):
    qubit_op=FermionicOperator(h1,h2).mapping('jordan_wigner')
    return qubit_op

### Computing Energy gradient

In [4]:
def gradient(data):
    qubit_H,PauliOp,state=data
    M=op_converter.to_matrix_operator(commutator(qubit_H,PauliOp)).matrix
    return numpy.abs(numpy.conjugate(state).dot((M.dot(state))))

### Computing Energy Landscape

In [10]:
def Energy(params,H=None):
    backend=Aer.get_backend('statevector_simulator')
    circ=var_form_base.construct_circuit(parameters=params)
    stateVector_0=execute(circ,backend,shots=1024).result().get_statevector()
    if (H is None):
        E=(numpy.conjugate(stateVector_0)@Hmat@stateVector_0).real
    else:
        E=(numpy.conjugate(stateVector_0)@H@stateVector_0).real
    return E

### Expanding ansatz commutator pool/qubit-UCCSD pool

In [11]:
def ansatzExpander(ordChoice=None,poolChoice=None):
    def returnStringCoords(label):
        label=list(label)
        coords=[]
        for i in range(len(label)):
            if ((label[i]=='X') or (label[i]=='Y') or (label[i]=='Z')):
                coords.append(i)
        return coords
    def checkNonOverlappingMembers(label,ExcOps):
        Arr1=numpy.array(returnStringCoords(label))
        Arr2=numpy.array(returnStringCoords(ExcOps[-1]))
        A=numpy.intersect1d(Arr1,Arr2)
        return len(A)!=0
    def chooseOp(indices,pool):
        if (len(ExcOps)!=0):
            labelsOfChosenStrings=[checkNonOverlappingMembers(pool[indices[i]].paulis[0][1].to_label(),ExcOps) for i in range(len(indices))]
            chosenlabels=[pool[indices[i]].paulis[0][1].to_label() for i in range(len(labelsOfChosenStrings))]
            if False in labelsOfChosenStrings:
                ind=labelsOfChosenStrings.index(False)
                PauliOp=pool[indices[ind]]
            else:    
                PauliOp=pool[indices[0]] 
        else:
            PauliOp=pool[indices[0]]
        return PauliOp 
    backend=Aer.get_backend('statevector_simulator')
    circ=var_form_base.construct_circuit(parameters=params)
    state=execute(circ,backend,shots=1024).result().get_statevector()
    if(poolChoice==None):
        grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(gradient)((qubitH,commutator_pool[i],state)) for i in range(len(commutator_pool))))#qubit_pool_UCCSD))))#
        maxgrad=max(grads)
        flag=0
        neighborhood=0.2
        if (maxgrad<0.2):
            grads1=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(gradient)((qubitH,commutator_pool_2[i],state)) for i in range(len(commutator_pool_2))))
            if maxgrad<max(grads1):
                maxgrad=max(grads1)
                flag=1

        if flag==0:
            indices= numpy.where(numpy.abs(numpy.array(grads)-maxgrad)<neighborhood*maxgrad)[0]
            PauliOp=chooseOp(indices,commutator_pool) if ordChoice=='1' else commutator_pool[numpy.array(grads)==maxgrad][0]
        else:   
            indices= numpy.where(numpy.abs(numpy.array(grads1)-maxgrad)<neighborhood*maxgrad)[0]
            PauliOp=chooseOp(indices,commutator_pool_2) if ordChoice=='1' else commutator_pool_2[numpy.array(grads1)==maxgrad][0]   
    elif (poolChoice=='1'):
        grads=numpy.array(Parallel(n_jobs=7,verbose=2)(delayed(gradient)((qubitH,qubit_pool_UCCSD[i],state)) for i in range(len(qubit_pool_UCCSD))))
        maxgrad=max(grads)
        neighborhood=0.5
        indices= numpy.where(numpy.abs(numpy.array(grads)-maxgrad)<neighborhood*maxgrad)[0]
        PauliOp=chooseOp(indices,qubit_pool_UCCSD) if ordChoice=='1' else qubit_pool_UCCSD[numpy.array(grads)==maxgrad][0]
    print("max. gradient",maxgrad)    
    return PauliOp

In [1]:
def prepare_givens_rotated_state(givens):
        N=4
        circ = QuantumCircuit(2*N)
        # Fill first N_f orbitals for each spin
        for i in range(N_f):
            circ.x(i)
            circ.x(i+N)
        for rots in givens:
            for tup in rots:
                #for spin down
                spin=0
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
                #for spin up
                spin=1
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
        final_state_vector=execute(circ,backend,shots=1024).result().get_statevector()      
        return circ,final_state_vector

# Basis Rotation to MO representation

In [2]:
def prepare_given_operator(givens):
    def X(n):
        S=['I']*num_qubits
        S[n]='X'
        S=S[::-1]
        return WeightedPauliOperator([[1,Pauli.from_label(''.join(S))]])
    def CX(n1,n2):
        S=['I']*num_qubits
        S1=S.copy()
        S1[n1]='Z'
        S1=S1[::-1]
        S2=S.copy()
        S2[n2]='X'
        S2=S2[::-1]
        S3=S.copy()
        S3[n1]='Z'
        S3[n2]='X'
        S3=S3[::-1]
        newS=((WeightedPauliOperator([[0.5,Pauli.from_label(''.join(S))]]).add(WeightedPauliOperator([[0.5,Pauli.from_label(''.join(S1))]]))).add(WeightedPauliOperator([[0.5,Pauli.from_label(''.join(S2))]]))).add(WeightedPauliOperator([[-0.5,Pauli.from_label(''.join(S3))]]))
        return newS
    def CRY(theta,n1,n2):
        c1=(1+numpy.cos(theta/2))/2
        c2=(1-numpy.cos(theta/2))/2
        c3=numpy.sin(theta/2)/2.
        S=['I']*num_qubits
        Op1=WeightedPauliOperator([[c1,Pauli.from_label(''.join(S))]])
        S1=S.copy()
        S1[n1]='Z'
        S1=S1[::-1]
        Op2=WeightedPauliOperator([[c2,Pauli.from_label(''.join(S1))]])
        S2=S.copy()
        S2[n2]='Y'
        S2=S2[::-1]
        Op3=WeightedPauliOperator([[-1j*c3,Pauli.from_label(''.join(S2))]])
        S3=S.copy()
        S3[n2]='Y'
        S3[n1]='Z'
        S3=S3[::-1]
        Op4=WeightedPauliOperator([[1j*c3,Pauli.from_label(''.join(S3))]])
        Op=((Op1.add(Op2)).add(Op3)).add(Op4)
        return Op
    def RZ(phi,n):
        c1=numpy.cos(phi/2)
        c2=-numpy.sin(phi/2)
        S=['I']*num_qubits
        S1=S.copy()
        S1[n]='Z'
        S1=S1[::-1]
        Op=WeightedPauliOperator([[c1,Pauli.from_label(''.join(S))]]).add(WeightedPauliOperator([[c2*1j,Pauli.from_label(''.join(S1))]]))
        return Op
    N_f=2
    N=num_sites
    S=WeightedPauliOperator([[1,Pauli.from_label('I'*num_qubits)]])
    # Fill first N_f orbitals for each spin  
    for rots in tqdm(givens):
        for tup in rots:
            for spin in range(2):
                S=CX(tup[1]+N*spin,tup[0]+N*spin).multiply(S)
                S=CRY(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin).multiply(S)
                S=CX(tup[1]+N*spin, tup[0]+N*spin).multiply(S)
                S=RZ(tup[3],tup[1]+N*spin).multiply(S)
    return S

In [3]:
def HF_init_state(U,return_MO_rotnOp=False):
    backend=Aer.get_backend('statevector_simulator')
    N=4
    # Qiskit implementation of Givens rotation
    def prepare_givens_rotated_state(givens):
        circ = QuantumCircuit(2*N)
        # Fill first N_f orbitals for each spin
        for i in range(N_f):
            circ.x(i)
            circ.x(i+N)
        for rots in givens:
            for tup in rots:
                #for spin down
                spin=0
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
                #for spin up
                spin=1
                circ.cnot(tup[1]+N*spin,tup[0]+N*spin)
                circ.cry(-2*tup[2],tup[0]+N*spin, tup[1]+N*spin)
                circ.cnot(tup[1]+N*spin, tup[0]+N*spin)
                circ.rz(tup[3],tup[1]+N*spin)
        final_state_vector=execute(circ,backend,shots=1024).result().get_statevector()      
        return circ,final_state_vector
    def cost_fn(angles):
        givens=[((1, 2, angles[0], 0.0),), ((0, 1, angles[1], 0.0), (2, 3, angles[2], 0.0)), ((1, 2, angles[3], 0.0),)]  
        #c=Givens_rot_circuit(givens)
        #state = QuantumState(N*2)
        #c.update_quantum_state(state)
        #state_vector=numpy.array(state.get_vector())
        c,state=prepare_givens_rotated_state(givens)
        E=numpy.real(numpy.dot(numpy.dot(numpy.conjugate(state),Hmat),state))
        return E
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot        
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites] \
            = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix#qubitH.to_matrix(massive=True)
    E,V=numpy.linalg.eigh(eg_h1)
    energy = 2*sum(E[:int(N/2)])
    psi = V[:,:int(N/2)]
    Q = numpy.transpose(numpy.conjugate(psi))
    N_f=len(Q)
    givens_init = slater_determinant_preparation_circuit(Q)
    params_init=[]
    for k in range(len(givens_init)):
        for l in range(len(givens_init[k])):
            params_init.append(givens_init[k][l][2])        
    res = scipy.optimize.minimize(cost_fn, params_init, bounds=[[-numpy.pi,numpy.pi]]*4,method='L-BFGS-B')
    print("Final Hartree Fock Energy",res['fun'])
    #final slater determinant state vector obtained using quantum variational Hartree Fock using Given Rotations
    params_fin=res['x']
    #Final Givens Rotation Circuit obtained after implementing L-BFGS-B algorithm
    givens_fin=[((1, 2, params_fin[0], 0.0),), ((0, 1, params_fin[1], 0.0), (2, 3, params_fin[2], 0.0)), ((1, 2, params_fin[3], 0.0),)]
    HFcirc,HFstateVec=prepare_givens_rotated_state(givens_fin)
    if return_MO_rotnOp==True:
        givensOp=prepare_given_operator(givens_fin)
        return HFcirc,HFstateVec,givensOp
    return HFcirc,HFstateVec

In [4]:
def MO_Rep_Op(MO_rep,qubit_H):
    MOrep=[]
    MOrep_dag=[]

    
    for i in tqdm(range(len(MO_rep.paulis))):
        print(i)
        if(numpy.abs(MO_rep.paulis[i][0])>1e-7):
            MOrep.append([MO_rep.paulis[i][0],MO_rep.paulis[i][1]])
            label=MO_rep.paulis[i][1].to_label()
            label=''.join(label.split()[::-1])
            MOrep_dag.append([numpy.conjugate(MO_rep.paulis[i][0]),Pauli.from_label(label)])
            
    MO1=WeightedPauliOperator(MOrep)
    MO2=WeightedPauliOperator(MOrep_dag)        
    qubit_H_MO=MO2.multiply(qubitH.multiply(MO1))
    qubit_H_MO_1=WeightedPauliOperator([[0,Pauli.from_label('I'*num_qubits)]])
    for i in tqdm(range(len(qubit_H_MO.paulis))):
        print(i)
        if(numpy.abs(qubit_H_MO.paulis[i][0])>1e-7):
            qubit_H_MO_1.add(WeightedPauliOperator([[qubit_H_MO.paulis[i][0],qubit_H_MO.paulis[i][1]]]))
    qubit_H_MO=qubit_H_MO_1
    return qubit_H_MO

In [5]:
def egBandHamiltonian(U):
    #Getting chemical Potential for Half-Filling
    with open('../chem_pot_for_Half_Fill.txt','r') as f:
        lines=f.readlines()[1:]
        for line in lines:
            elems=line.split()
            if int(elems[0])==U:
                muHalf=float(elems[1]) #Chem Pot for a given Hubbard U
    #Getting the one body and two body interaction vertexes
    with open('../'+str(U)+'/v1e.dat','r') as f:
            lines=f.readlines()[1:]
            num_sites=4
            chem_pot=numpy.zeros((2*num_sites,2*num_sites))
            eg_h1=numpy.zeros((2*num_sites,2*num_sites))
            for line in lines:
                elems=line.split()
                eg_h1[int(elems[0])][int(elems[1])]=float(elems[2])
                eg_h1[int(elems[0])+num_sites][int(elems[1])+num_sites]=float(elems[2])
            for i in range(2*num_sites):
                chem_pot[i][i]=-muHalf
            eg_h1=eg_h1+chem_pot       
    with open('../'+str(U)+'/v2e.dat','r') as f:
        num_sites=4
        eg_h2=numpy.zeros((2*num_sites,2*num_sites,2*num_sites,2*num_sites))
        for line in f:
            if "#" in line:
                continue
            line = line.split()
            i,j,k,l = map(int, line[:4])
            val = float(line[4])
            eg_h2[i,j,k,l] = eg_h2[i+num_sites,j+num_sites,k,l] = eg_h2[i,j,k+num_sites,l+num_sites] \
            = eg_h2[i+num_sites,j+num_sites,k+num_sites,l+num_sites] = 0.5*val  # convention with 0.5 factor included.
    qubitH=qubitOp(eg_h1,eg_h2)
    Hmat=op_converter.to_matrix_operator(qubitH).dense_matrix
    w,v=numpy.linalg.eigh(Hmat)
    Eg=w[0]
    state_g=v[:,0]
    return qubitH,Hmat,Eg,state_g

In [6]:
def InitState(U,choice=None):
    if choice=='1':
        qc=QuantumCircuit(8)
        qc.x(0)
        qc.x(1)
        qc.x(4)
        qc.x(5)
        var_form_base=UCCSD(8,4,initial_state=qc,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
    else:
        HFcirc,HFstate=HF_init_state(U)
        var_form_base=UCCSD(8,num_particles=4, initial_state=HFcirc,qubit_mapping='jordan_wigner',two_qubit_reduction=False)
    var_form_base.manage_hopping_operators()
    return var_form_base